In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from matplotlib import colors
import random

In [2]:
df = pd.read_csv('agent_1_data.csv')

In [3]:
c = []
x_train = []
for i in range(len(df)):
    arr = np.fromstring(df['x'][i].replace('[', '').replace(']', ''), dtype=int, sep='\n').reshape(3,3)
    arr = arr/arr.max()
    c.append(np.asarray(arr))
x_train = np.asarray(c)
x_train = np.asarray(x_train)
x_train.shape

(29162, 3, 3)

In [4]:
out = []
for i in range(len(df)):
    a = df['y'][i].replace('[', '').replace(']', '').replace(',','').split()
    a = [int(m) for m in a]
    out.append(np.asarray(a))
y_train = out
y_train = np.asarray(out)
y_train

array([[1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       ...,
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0]])

In [5]:
y_train.shape

(29162, 4)

In [7]:
y_train.shape

(29162, 4)

In [8]:
digit_input = tf.keras.layers.Input( shape = (3,3,1) )
max_pool_2d = tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(1, 1), padding='valid')(digit_input)
flatten_image = tf.keras.layers.Flatten()( max_pool_2d )

dense_1 = tf.keras.layers.Dense( units = 128, activation = tf.nn.relu )( flatten_image )
dense_2 = tf.keras.layers.Dense( units = 128, activation = None )( dense_1 )
probabilities = tf.keras.layers.Dense( units = 4, activation =  tf.nn.softmax)( dense_2 )

In [9]:
model = tf.keras.Model( inputs = digit_input, outputs = probabilities )
model.compile( optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'] )

In [10]:
x_train = np.reshape( x_train, (-1, 3, 3, 1) )
x_test = np.reshape( x_test, (-1, 3, 3, 1) )
history = model.fit( x_train, y_train, epochs = 20 )

Epoch 1/20
912/912 [==============================] - 2s 1ms/step - loss: 1.3328 - accuracy: 0.4026
Epoch 2/20
912/912 [==============================] - 1s 1ms/step - loss: 1.3311 - accuracy: 0.4033
Epoch 3/20
912/912 [==============================] - 1s 1ms/step - loss: 1.3308 - accuracy: 0.4033
Epoch 4/20
912/912 [==============================] - 1s 1ms/step - loss: 1.3304 - accuracy: 0.4033
Epoch 5/20
912/912 [==============================] - 1s 1ms/step - loss: 1.3303 - accuracy: 0.4033
Epoch 6/20
912/912 [==============================] - 1s 1ms/step - loss: 1.3302 - accuracy: 0.4033
Epoch 7/20
912/912 [==============================] - 1s 1ms/step - loss: 1.3303 - accuracy: 0.4033
Epoch 8/20
912/912 [==============================] - 1s 1ms/step - loss: 1.3300 - accuracy: 0.4033
Epoch 9/20
912/912 [==============================] - 1s 1ms/step - loss: 1.3301 - accuracy: 0.4033
Epoch 10/20
912/912 [==============================] - 1s 1ms/step - loss: 1.3301 - accuracy: 0.4033

In [ ]:
x_test = [[3, 3, 3],
       [3, 3, 3],
       [1, 4, 3]]
x_test = np.asarray(x_test)

y_test = np.asarray([0, 0, 0, 1])

In [ ]:
# calculate the validation loss
val_loss, val_accuracy = model.evaluate(x_test, np.asarray(y_test))
print(val_loss, val_accuracy)

In [ ]:
model.save('epic_num_reader2.model')

In [ ]:
new_model = tf.keras.models.load_model('epic_num_reader2.model')

In [ ]:
def get_children(x, y, n, visited_list):
    """
    This function is used to return children of a particular node
    x: row value of current node
    y: column value of current node
    n: size of maze
    gofn: g(n) value of current node
    visited_list: list of nodes that are already visited
    matrix: maze
    agent_matrix: matrix of the agent that holds the information of updated environment
    """
    allChildren = [(x+1,y),#down
                   (x-1,y),#up
                   (x,y+1),#right
                   (x,y-1)]#left
    if x+1 > n-1:
        allChildren.remove((x+1,y))
    if x-1 < 0:
        allChildren.remove((x-1,y))
    if y+1 > n-1:
        allChildren.remove((x,y+1))
    if y-1 < 0:
        allChildren.remove((x,y-1))
#     allChildren = [node for node in allChildren if node[:2] not in visited_list]

    return allChildren

In [ ]:
def populateMatrix(n,p):
    """
    This function returns randomly populated maze
    p: probabillity density of each cell
    n: size of the maze
    """
    global block_unblock_matrix
    np_array = np.random.rand(n*n)
    block = lambda x: 0 if x>=p else 1
    vectorized_block = np.vectorize(block)
    np_array = vectorized_block(np_array).reshape(n,n)
    np_array[0,0]=0
    np_array[n-1,n-1] =0
    block_unblock_matrix = np.asmatrix(np_array)

    ############testing on hard coded matrix ###########
#     block_unblock_matrix = [[0, 0, 0, 0, 0],
#             [0, 0, 0, 0, 0],
#             [0, 0, 0, 0, 0],
#             [0, 1, 0, 0, 0],
#             [1, 0, 0, 1, 0]]
#     ######################################
#     populateMatrix(n,p)
    block_unblock_matrix = np.matrix(block_unblock_matrix)
    plt.spy(block_unblock_matrix)
    return block_unblock_matrix


In [ ]:
maze_size = 30
agent_grid_world = np.full((maze_size, maze_size),3)
agent_grid_world = np.asmatrix(agent_grid_world)
full_grid_world = populateMatrix(maze_size, 0.2)
plt.spy(full_grid_world)

In [ ]:
def next_move(current_node, predictions, full_grid_world, visited_list, maze_size, trajectory, flag, agent_grid_world):
    """
    Based on prediction made by the Neural Network, this function decides the next move of the agent
    current_node: The node that the agent is currently present in
    predictions: The output given by Neural Network
    full_grid_world: The actual maze/gridworld
    visited_list: list of nodes that have been visited by the agent
    maze_size: Size of the gridworld
    trajectory: The path that agent took to reach the goal
    flag: status to check if the current node is blocked or not
    agent_grid_world: The agent-discovered portion of the full grid world
    """
    message = ''
    indices = zip(np.where(predictions[0] == predictions[0].max()))

    for row in indices:
        print(row)
        if row[0][0] == 0:
            message = 'up'
            next_node = (current_node[0] - 1, current_node[1])
        elif row[0][0] == 1:
            message = 'down'
            next_node = (current_node[0] + 1, current_node[1])
        elif row[0][0] == 2:
            message = 'left'
            next_node = (current_node[0], current_node[1] - 1)
        elif row[0][0] == 3:
            message = 'right'
            next_node = (current_node[0], current_node[1] + 1)
    
    try:
        if agent_grid_world[next_node] == 2 or next_node[0] > maze_size-1 or next_node[1] > maze_size-1 or next_node in visited_list:
            children = get_children(current_node[0], current_node[1], maze_size, visited_list)
            i = random.randint(0,len(children)-1)
            if (next_node[0], next_node[1]) == (current_node[0] - 1, current_node[1]):
                message = 'up'
            elif (next_node[0], next_node[1]) == (current_node[0] + 1, current_node[1]):
                message = 'down'
            elif (next_node[0], next_node[1]) == (current_node[0] - 1, current_node[1]):
                message = 'left'
            elif (next_node[0], next_node[1]) == (current_node[0] + 1, current_node[1]):
                message = 'right'

            return children[i], message
    except Exception as err:
        children = get_children(current_node[0], current_node[1], maze_size, visited_list)
        i = random.randint(0,len(children)-1)
    
        if (children[0], children[1]) == (current_node[0] - 1, current_node[1]):
                message = 'up'
        elif (children[0], children[1]) == (current_node[0] + 1, current_node[1]):
            message = 'down'
        elif (children[0], children[1]) == (current_node[0] - 1, current_node[1]):
            message = 'left'
        elif (children[0], children[1]) == (current_node[0] + 1, current_node[1]):
            message = 'right'

        return children[i], message
    
    return next_node, message

In [ ]:
def get_neighbors(row, col, n):
    """
    Returns the immediate neighbors of the node (row, col)
    n: Maze size
    """
    dir = [[-1, -1], [-1, 0],[-1, 1], [0, -1], [0, 0],  [0, 1], [1, -1], [1, 0], [1, 1]]
    friend = []
    count = 0
    for x, y in dir:
        row_new = row + x
        col_new = col + y
        if (row_new>=0 and row_new<=(n-1)) and (col_new>=0 and col_new<=(n-1)):
            count += 1
            friend.append((row_new, col_new))
    return count, friend

In [ ]:
def get_window(current_node, agent_grid_world, full_grid_world):
    """
    Since we use the local information, this function returns the local 3x3 grid surrounding the agent.
    current_node: The node that the agent is currently present in
    agent_grid_world: The agent-discovered portion of the full grid world
    full_grid_world: The actual maze/gridworld
    """
    a = []
    b = []
    count, neighbors = get_neighbors(current_node[0],current_node[1], maze_size)
    if len(neighbors) == 4:
        if current_node == (0,0):
            count, neighbors = get_neighbors(1,1, maze_size)
        elif current_node == (0,maze_size-1):
            count, neighbors = get_neighbors(1,maze_size-2, maze_size)
        elif current_node == (maze_size-1,0):
            count, neighbors = get_neighbors(maze_size-2, 1, maze_size)
        elif current_node == (maze_size-1,maze_size-1):
            count, neighbors = get_neighbors(maze_size-2,maze_size-2, maze_size)
    elif len(neighbors) == 6:
        if current_node[0] == 0:
            count, neighbors = get_neighbors(current_node[0] + 1,current_node[1], maze_size)
        elif current_node[0] == maze_size-1:
            count, neighbors = get_neighbors(current_node[0] - 1,current_node[1], maze_size)
        elif current_node[1] == 0:
            count, neighbors = get_neighbors(current_node[0],current_node[1] + 1, maze_size)
        elif current_node[1] == maze_size-1:
            count, neighbors = get_neighbors(current_node[0],current_node[1] - 1, maze_size)
    for i in neighbors:
        a.append(agent_grid_world[i])
        b.append(full_grid_world[i])

    window = np.asarray(a).reshape(3,3)

    return window

In [ ]:
goal_reached = False
new_model = tf.keras.models.load_model('epic_num_reader2.model')

start = (0,0)
goal = (maze_size-1, maze_size-1)
agent_grid_world.itemset(start, 4)
current_node = start
visited_list = []
trajectory = []

while goal_reached == False:
    trajectory.append(current_node)
    if current_node == goal:
        agent_grid_world.itemset(goal, 1)
        print('Goal reached!!!')
        goal_reached = True
        break

    if current_node not in visited_list:
        visited_list.append(current_node)

    x_test = get_window(current_node, agent_grid_world, full_grid_world)
    x_test = np.asarray(x_test)

    predictions = new_model.predict(np.asarray([x_test]))

    if full_grid_world.item(current_node) == 1:
        
        agent_grid_world.itemset(current_node, 2)
        flag = 'blocked'
    else:
        agent_grid_world.itemset(current_node, 1)
        flag = 'free'
    
    if flag == 'free':
        try:
            next_node, message = next_move(current_node, predictions, full_grid_world, visited_list, maze_size, trajectory, flag, agent_grid_world)
            current_node = next_node
        except Exception as err:
            print(err)
            print('No path to goal')
            break
        if message == "Can't reach goal":
            print('Cannot reach goal')
            break

    if flag == 'blocked':
        current_node = trajectory[-2]

    agent_grid_world.itemset(current_node, 4)

    colormap = colors.ListedColormap(["green","white"])
    plt.imshow(agent_grid_world, cmap=colormap)
    plt.show()
print("The length of trajectory is:", len(trajectory))


In [ ]:
plt.spy(full_grid_world)

In [ ]:
colormap = colors.ListedColormap(["green","white"])
plt.imshow(agent_grid_world, cmap=colormap)
plt.show()